In [149]:
import pandas as pd
import pyodbc

conn = pyodbc.connect("""DRIVER={SQL Server};
                            SERVER=DMT-KRK-L111;
                            DATABASE=DBTest;
                            Trusted_Connection=yes""") 
query = "SELECT TOP 1 * FROM UserInput"
df = pd.read_sql(query, conn)
material = pd.read_excel('Frezowanie rowkow.xlsx',sheet_name='Material')

Mat = df['Mat']
Mat_Selected = material[material['Mat_PN']==Mat.iloc[0]]
df['Rm'] = Mat_Selected['Rm'].iloc[0]
df['HB'] = Mat_Selected['HB'].iloc[0]
Rm = df['Rm']
HB = df['HB']

In [150]:
import pickle
with open('modelRF[Vc]', 'rb') as f:
    model1 = pickle.load(f)

with open('modelRF[pz]', 'rb') as f:
    model2 = pickle.load(f)
    
with open('modelRF[ks]', 'rb') as f:
    model3 = pickle.load(f)    

with open('modelRF[ap]', 'rb') as f:
    model4 = pickle.load(f)    

In [151]:
R = df['R']
G = df['G']
L = df['L']

Vc = model1.predict(df[['L', 'R', 'G', 'Rm', 'HB']])
pz = model2.predict(df[['L', 'R', 'G', 'Rm', 'HB']])
ks = model3.predict(df[['L', 'R', 'G', 'Rm', 'HB']])
ap = model4.predict(df[['L', 'R', 'G', 'Rm', 'HB']])

In [152]:
import math
n = Vc*1000/((math.pi)*df['R'])
f = pz * 2
Vf = f * n
Amax = 2 * pz * math.sqrt(ap / R)
IL = math.ceil((G + 1.5) / ap)
Tg = IL * L / Vf
Fs = ks * ap * f 
M = Fs * R / 2000
Ps = M * n /9554
Pe = Ps/0.6

In [153]:
newSeries = pd.Series(data=(Mat.iloc[0], Rm.iloc[0],HB.iloc[0],R.iloc[0],G.iloc[0],L.iloc[0],Vc[0],Vf.iloc[0],Ps.iloc[0],n.iloc[0],f[0],pz[0],ap[0],Amax[0],ks[0],Fs[0],M.iloc[0],Pe.iloc[0],IL,Tg.iloc[0]), index=('Mat','Rm','HB','R','G','L','Vc','Vf','Ps','n','f'
                               ,'pz','ap','Amax','ks','Fs','M','Pe', 'IL','Tg'))
df1 = df.append(newSeries, ignore_index=True)

In [154]:
df1

,R,G,L,Mat,Rm,HB,Amax,Fs,IL,M,Pe,Ps,Tg,Vc,Vf,ap,f,ks,n,pz
0,32.0,9.0,78.0,30G2,800,207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32.0,9.0,78.0,30G2,800,207,0.078957,3816.00872,4.0,61.05614,1.589221,0.953533,7.631197,15.0,40.884803,2.657,0.274013,5241.4,149.207759,0.137006
